In [1]:
##  LIBRERIAS  
import os
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from tqdm.notebook import trange, tqdm
import time
import math

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon

from mpl_toolkits import mplot3d
%matplotlib inline

pd.set_option('display.max_columns', 50)

In [2]:
## LECTURA ... Df Parametros
par_file_name="CLAG1.par"

#print(open(par_file_name, "r").read())
open(par_file_name, "r").close()
l=[re.split("[']", x) for x in open(par_file_name, "r")]
#print(x)
open(par_file_name, "r").close()            #cerrar archivo 
## diccionario de parámetros
param={}
param_l=[]
paran_v=[]

## OJO!! 
# Separador no es el mismo en todos los SO -->  Usar OS

## header:          PARÁMETRO 1
param[1]=l[0][0].strip()
param_l.append(l[0][1].strip().replace(os.sep,""))
## input file:          PARÁMETRO 2
param[2]=l[1][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[1][1].strip().replace(os.sep,""))
## columns for data:          PARÁMETRO 3
param[3]=[int(x) for x in l[2][0].strip().split(",")]
param_l.append(l[2][1].strip().replace(os.sep,"")[0:16])
## initial search volume:          PARÁMETRO 4
param[4]=[float(x) for x in l[3][0].strip().split(",")]
param_l.append(l[3][1].strip().replace(os.sep,""))
## Number of Risk Thresholds:          PARÁMETRO 5
param[5]=[int(x) for x in l[4][0].strip().split(",")]
param_l.append(l[4][1].strip().replace(os.sep,"")[0:37])
## Selected risk thresholds:          PARÁMETRO 6
param[6]=[float(x) for x in l[5][0].strip().split(",")]
param_l.append(l[5][1].strip().replace(os.sep,""))
## Number of data in local volume:          PARÁMETRO 7
param[7]=[int(x) for x in l[6][0].strip().split(",")]
param_l.append(l[6][1].strip().replace(os.sep,""))
## Number of units:          PARÁMETRO 8
param[8]=[int(x) for x in l[7][0].strip().split(",")]
param_l.append(l[7][1].strip().replace(os.sep,""))
## Unit names:          PARÁMETRO 9
param[9]=[int(x) for x in l[8][0].strip().split(",")]
param_l.append(l[8][1].strip().replace(os.sep,""))
## not consider values less than:          PARÁMETRO 10
param[10]=[int(x) for x in l[9][0].strip().split(",")]
param_l.append(l[9][1].strip().replace(os.sep,""))
## output files name:          PARÁMETRO 11
param[11]=l[10][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[10][1].strip().replace(os.sep,""))
## number of zones:          PARÁMETRO 12
param[12]=[int(x) for x in l[11][0].strip().split(",")]
param_l.append(l[11][1].strip().replace(os.sep,""))
## zone identifiers:          PARÁMETRO 13
param[13]=[int(x) for x in l[12][0].strip().split(",")]
param_l.append(l[12][1].strip().replace(os.sep,""))
## azimuth of search orientation per zone :          PARÁMETRO 14
param[14]=[int(x) for x in l[13][0].strip().split(",")]
param_l.append(l[13][1].strip().replace(os.sep,""))
## Anisotropy ratios:          PARÁMETRO 15
param[15]=[float(x) for x in l[14][0].strip().split(",")]
param_l.append(l[14][1].strip().replace(os.sep,""))
## Exclude data from the same drillhole:          PARÁMETRO 16
param[16]=[int(x) for x in l[15][0].strip().split(",")]
param_l.append(l[15][1].strip().replace(os.sep,""))
## Option to weight data by search expansion:          PARÁMETRO 17
param[17]=[int(x) for x in l[16][0].strip().split(",")]
param_l.append(l[16][1].strip().replace(os.sep,""))

df_parametros = pd.DataFrame(list(zip(list(range(1,18)), param_l, list(param.values()))),
                            columns=['Orden','Descripcion', 'Valor']
                        )


df_parametros






,Orden,Descripcion,Valor
0,1,header,Capping Test Copper Using BHID
1,2,input file,CLAG_MR2022_2.csv
2,3,columns for data,"[1, 2, 3, 4, 6, 5, 7, 8]"
3,4,initial search volume,"[30.0, 30.0, 30.0]"
4,5,Number of Risk Thresholds to consider,[6]
5,6,selected risk thresholds,"[0.1, 0.125, 0.15, 0.175, 0.2, 0.225]"
6,7,number of data in local volume,[21]
7,8,number of units,[2]
8,9,unit names,"[31, 32]"
9,10,do not consider values less than,[0]


In [3]:
##   Lectura CSV compositos .. validar si hay nulos o no

df_CLAG_MR_2022 = pd.read_csv(param[2])
df_CLAG_MR_2022.head(2)
## Requerimiento: No NaN ni missing values
'''
Algoritmo localcapping no admite valores nulos ni missings
función missing_val(df) identifica casos missing y nulos
'''

def missing_val(df):
    cero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mz_table = pd.concat([cero_val, mis_val], axis=1)
    mz_table = mz_table.rename(
               columns = {0 : 'Valores  Cero', 1 : 'Valores Missing'})
    mz_table['Total Cero + Missing'] = mz_table['Valores  Cero'] + mz_table['Valores Missing']
    mz_table = mz_table[mz_table.iloc[:,1] != 0].round(1)
    print ("Df Columnas: " + str(df.shape[1]) + " Df Filas: " + str(df.shape[0]) + " \n"      
            "Columnas con Ceros / Missing: " + str(mz_table.shape[0]))
    return mz_table

## 
missing_val(df_CLAG_MR_2022)  




Df Columnas: 8 Df Filas: 4255 
Columnas con Ceros / Missing: 0


,Valores Cero,Valores Missing,Total Cero + Missing


In [4]:
# ACONDICIONAR DATAFRAME


'''
Este ordenamiento no limita la seleccion decolumnas por nombre
Hace seleccion en funcion del orden inicial de columnas
'''
df_CLAG_MR_2022_R=df_CLAG_MR_2022[[df_CLAG_MR_2022.iloc[:,0].name,
                                   df_CLAG_MR_2022.iloc[:,1].name,
                                   df_CLAG_MR_2022.iloc[:,2].name,
                                   df_CLAG_MR_2022.iloc[:,3].name,
                                   df_CLAG_MR_2022.iloc[:,4].name,
                                   df_CLAG_MR_2022.iloc[:,5].name,
                                   df_CLAG_MR_2022.iloc[:,6].name,
                                   df_CLAG_MR_2022.iloc[:,7].name]]

df_CLAG_MR_2022=df_CLAG_MR_2022_R

df_CLAG_MR_2022.head(2)


##  $$$  INICILIZAR VALORES PARA 6 UMBRALES
##  y    #ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6
#         ,AvgSin,PROP_12,Num_Data

df_CLAG_MR_2022["CAPVAL1_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED1_PY"]=0
df_CLAG_MR_2022["CAPVAL2_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED2_PY"]=0
df_CLAG_MR_2022["CAPVAL3_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED3_PY"]=0
df_CLAG_MR_2022["CAPVAL4_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED4_PY"]=0
df_CLAG_MR_2022["CAPVAL5_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED5_PY"]=0
df_CLAG_MR_2022["CAPVAL6_PY"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED6_PY"]=0

df_CLAG_MR_2022["AvgSin_PY"]=0
df_CLAG_MR_2022["PROP_12_PY"]=0
df_CLAG_MR_2022["Num_Data_PY"]=0

df_CLAG_MR_2022["EXPAN_PY"]=0

df_CLAG_MR_2022["EVALUADO_PY"]=0

df_CLAG_MR_2022["LEY_INDEX_PY"]=0
df_CLAG_MR_2022["INDEX_PY"]=0
df_CLAG_MR_2022["SECUENTIAL_INDEX_PY"]=0
df_CLAG_MR_2022["ROCA_PY"]=0
df_CLAG_MR_2022["ZONA_PY"]=0


df_CLAG_MR_2022_ORIG=df_CLAG_MR_2022

df_CLAG_MR_2022.head(2)






,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1_PY,CAPPED1_PY,CAPVAL2_PY,CAPPED2_PY,CAPVAL3_PY,CAPPED3_PY,CAPVAL4_PY,CAPPED4_PY,CAPVAL5_PY,CAPPED5_PY,CAPVAL6_PY,CAPPED6_PY,AvgSin_PY,PROP_12_PY,Num_Data_PY,EXPAN_PY,EVALUADO_PY,LEY_INDEX_PY,INDEX_PY,SECUENTIAL_INDEX_PY,ROCA_PY,ZONA_PY
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0,0,0,0,0,0,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0,0,0,0,0,0,0,0,0,0


In [6]:

df_CLAG_MR_2022.loc[[73]]
#df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==6586]

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1_PY,CAPPED1_PY,CAPVAL2_PY,CAPPED2_PY,CAPVAL3_PY,CAPPED3_PY,CAPVAL4_PY,CAPPED4_PY,CAPVAL5_PY,CAPPED5_PY,CAPVAL6_PY,CAPPED6_PY,AvgSin_PY,PROP_12_PY,Num_Data_PY,EXPAN_PY,EVALUADO_PY,LEY_INDEX_PY,INDEX_PY,SECUENTIAL_INDEX_PY,ROCA_PY,ZONA_PY
73,6586,3185.3567,4329.8712,4116.0064,0.828571,31,41,A055,0.828571,0,0.828571,0,0.828571,0,0.828571,0,0.828571,0,0.828571,0,0,0,0,0,0,0,0,0,0,0


In [43]:
N=9336
df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==N].index[0]
#df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==6586]

197

In [44]:
rocks=df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==N].NBKRT
zonas=df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==N].AG_SUBZONAS

In [45]:
zonas=df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==N].AG_SUBZONAS
zonas


197    41
Name: AG_SUBZONAS, dtype: int64

In [46]:
df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 0] == N)].T

,197
COUNT,9336
mid_x,3724.84
mid_y,4797.7
mid_z,4196.19
Ag_ppm_BESTEL,95.25
NBKRT,32
AG_SUBZONAS,41
HOLEID,A080
CAPVAL1_PY,95.25
CAPPED1_PY,0


In [47]:


parametro_expansion=0.25
expansiones_max=12
expansiones_realizadas=0
round_hemi_accuracy=2

max_compositos_interiores=19  # ( restar 1 al numero objetivo)

freno_1_composito_x_roca=0  ###  SOLO LEER UN SOLO COMPOSITO POR COMBINACION ROCA/ZONA
freno_2_roca_x_zona=0  ###  SOLO LEER UNA SOLA ROCA POR ZONA
freno_3_zona=0    ### SOLO LEER UNICA ZONA

iterador=0      ##  PARA SIGNAR RATIO x ZONA

composito_counts=0

flag_holeid=df_parametros.Valor[15][0]

#for zone in df_parametros.Valor[12][:1]:     ## borrar   [:2]
zonas = df_parametros.Valor[12]
for zone in zonas:     #################################  FIJO PARA PRUEBA  BORRAR
    
    print("\n\n")
    print("zone:",zone)
    print(" ") 
    #  df_parametros.Valor[13]         ## LISTA DE AZIMUTH
    print("azimuth:",df_parametros.Valor[13][iterador])
    azimuth=df_parametros.Valor[13][iterador]
    cartesiana=0          ## <------ 

    if azimuth < 90:
        cartesiana=90-azimuth 
    elif azimuth < 180:
        cartesiana=360-(azimuth-90) 
    elif azimuth < 270:
        cartesiana=270-(azimuth-180) 
    elif azimuth < 360:
        cartesiana=180-(azimuth-270) 
        
    print("cartesiana:",cartesiana)
    rad=math.radians(cartesiana)    ## <------  APLICAR ACA SIN COS etc etc    
    print("radian:",rad)
    #df_parametros.Valor[14]                            ## LISTA DE ANISOTROPIA 
    print("ratio anisotropia:",df_parametros.Valor[14][iterador])

    ratio=df_parametros.Valor[14][iterador]
    hemi_axis_x, hemi_axis_y, hemi_axis_z = [], [], []
    for i in range(expansiones_max):       #  30*30*30       30*75*30
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i))
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)*2)
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)*2)
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)*2)
        semi_x=(df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)
        semi_y=((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)
        semi_z=(df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)
        hemi_axis_x.append(round(semi_x,round_hemi_accuracy))
        hemi_axis_y.append(round(semi_y,round_hemi_accuracy))
        hemi_axis_z.append(round(semi_z,round_hemi_accuracy))

    ###for rock in df_parametros.Valor[8][:1]:## borrar   [3]
    #rocks=df_parametros.Valor[8]
    for rock in rocks:

        freno_1_composito_x_roca=0

        print("\n\n\n")
        print("    rock:",rock)
        print("    zone:",zone)
        print(" ")
        print("    parametros de la Roca ")
        print("    ratio anisotropia:",ratio)
        print("    azimuth: ",azimuth)
        print("    cartesiana: ",cartesiana)
        print("    radian: ",rad)

        for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 0] == N)].T:  ## ITERAR INDICES

            composito_counts=composito_counts+1

            print("\n")
            print("        index composito :",obs)
            print(" ")
            x=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,1][obs]
            y=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,2][obs]
            z=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,3][obs]
             
            print("        x: ",x)
            print("        y: ",y)
            print("        z: ",z)
            obs_holeid=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs][7]
            print(" ")
            print("        holeid: ",obs_holeid)
            # x_SI,y_SI= x - semi_x,y + semi_y        
            # x_II,y_II= x - semi_x,y - semi_y
            # x_SD,y_SD= x + semi_x,y + semi_y
            # x_ID,y_ID= x + semi_x,y - semi_y      
            # print("        SI:",x_SI,y_SI)
            # print("        II:",x_II,y_II)  
            # print("        SD:",x_SD,y_SD) 
            # print("        ID:",x_ID,y_ID)
            # s = gpd.GeoSeries([Polygon([(x_II,y_II), (x_SI, y_SI), (x_SD, y_SD), (x_ID, y_ID)])])      
            ## RESPETAR ORDEN II SI SD ID
            #s[0]
            #s.rotate(cartesiana, origin=(x,y))[0]
            # rx,ry = s.rotate(  cartesiana, origin=(x,y))[0].exterior.coords.xy
            # coords = np.dstack((rx,ry)).tolist()
            #print(coords)           
            # print("        Rotated coords: ")
            # print("        ",coords[0][0][0],coords[0][0][1])
            # print("        ",coords[0][1][0],coords[0][1][1])
            # print("        ",coords[0][2][0],coords[0][2][1])
            # print("        ",coords[0][3][0],coords[0][3][1])
            # print("        ",coords[0][4][0],coords[0][4][1])
            # rotado2d=s.rotate(cartesiana, origin=(x,y))[0]
            # index_inner_3d=[]## INDICES EN POLIGONO ROTADO 3D  
            index_inner_3d=[]## INDICES EN POLIGONO ROTADO 3D
            ley=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,4][obs]
            print("        ley composito : ",ley)

            for i in range(expansiones_max):
                print(" ")
                print(" ")
                print("            expansion: ",i)
                print(" ")
                '''
                print("            semi eje x:",hemi_axis_x[i])
                print("            semi eje y:",hemi_axis_y[i])
                print("            semi eje z:",hemi_axis_z[i])
                '''

                x_SI,y_SI= x - hemi_axis_x[i] , y + hemi_axis_y[i]       
                x_II,y_II= x - hemi_axis_x[i] , y - hemi_axis_y[i]
                x_SD,y_SD= x + hemi_axis_x[i] , y + hemi_axis_y[i]
                x_ID,y_ID= x + hemi_axis_x[i] , y - hemi_axis_y[i]  

                '''
                print("            SI:  x: ",x_SI,"  y:",y_SI)
                print("            II:  x: ",x_II,"  y:",y_II)  
                print("            SD:  x: ",x_SD,"  y:",y_SD) 
                print("            ID:  x: ",x_ID,"  y:",y_ID)
                '''

                s = gpd.GeoSeries([Polygon([(x_II,y_II), (x_SI, y_SI), (x_SD, y_SD), (x_ID, y_ID)])])      
                # RESPETAR ORDEN II SI SD ID
                #s[0]
                #s.rotate(cartesiana, origin=(x,y))[0]
                #rx,ry = s.rotate(  cartesiana, origin=(x,y))[0].exterior.coords.xy
                rotado2d=s.rotate(cartesiana, origin=(x,y))[0]
                rx,ry = rotado2d.exterior.coords.xy
                coords = np.dstack((rx,ry)).tolist()
                '''
                print("            Rotated coords: ")
                print("            x: ",coords[0][0][0]," y: ",coords[0][0][1])
                print("            x: ",coords[0][1][0]," y: ",coords[0][1][1])
                print("            x: ",coords[0][2][0]," y: ",coords[0][2][1])
                print("            x: ",coords[0][3][0]," y: ",coords[0][3][1])
                print("            x: ",coords[0][4][0]," y: ",coords[0][4][1])
                '''


                for obs_sub in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:

                    '''
                    print(" ")
                    print("                obs    : ",obs)
                    print("                obs_sub: ",obs_sub)
                    '''
                    sub_ley=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,4][obs_sub]
                    '''
                    print("                ley_sub: ",sub_ley)   
                    ''' 

                    x_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][1]
                    y_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][2]
                    '''
                    print("                x_sub: ",x_int)
                    print("                y_sub: ",y_int)
                    '''

                    composito_inter= gpd.GeoSeries([Point(x_int,y_int)])  ##  COMPONER coords del subcomposito

                    if rotado2d.intersects(composito_inter[0]) == True:
                        '''
                        print("                    SI 2D")
                        '''

                        z_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][3]
                        '''
                        print("                    z_sub: ",z_int)
                        print("                    z:     ",z)
                        '''
                        z_inner_diff=abs(z-z_int)
                        '''
                        print("                    dist z:",z_inner_diff)
                        print("                    dist max z:",hemi_axis_z[i])
                        '''

                        if z_inner_diff <= hemi_axis_z[i]:
                            '''
                            print("                        SI 3D")
                            '''
                            #index_inner_3d.append(obs_sub)
                            #print("                        index_inner_3d: ",index_inner_3d)

                            subobs_holeid=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][7]

                            '''
                            print("                        drillhole obs    : ",obs_holeid)
                            print("                        drillhole obs sub: ",subobs_holeid)
                            '''

                            if obs_sub not in index_inner_3d:
                                if flag_holeid==1:
                                    '''
                                    print("                            SI FLAG EXCLUIR SAME ID")
                                    '''
                                    if obs_holeid!=subobs_holeid:
                                        '''
                                        print("                            DISTINTO ID  -->  APPEND") 
                                        '''
                                        index_inner_3d.append(obs_sub)
                                else:
                                    index_inner_3d.append(obs_sub)

                    '''
                    else:
                        print("                    NO 2D") 
                    '''

                    '''
                    if len(index_inner_3d) > max_compositos_interiores:       ####  ROMPER APENAS ENCUENTRE 20
                        break
                    '''


                print("            ",index_inner_3d)
                print("            longitud index_inner:   ",len(index_inner_3d))
                #print("# expansion sub compositos:   ",i)
                ## NECESARIO ROMPER A ESTE NIVEL 
                if len(index_inner_3d) > max_compositos_interiores:   ###  ROMPER EXPANSION
                    print("# expansion break bucles expansivos:   ",i)
                    break   ###  ROMPER EXPANSION

                # if i > 8:   ###  ROMPER EXPANSION por numero de expansiones
                #     break   ###  ROMPER EXPANSION
            print(" ")
            print("verificar lista de busqueda...")
            print("longitud inder_inner fuera del bucle    ",len(index_inner_3d))
 
            print("indice composito en evaluacion -- NO SUB--   ",obs)
            index_inner_3d.append(obs)
            print("lista final de compositos SUB asociados a composito en evaluacion + composito en evaluacion")
            print(index_inner_3d)
            print("longitud final index  ",len(index_inner_3d))


            print(" ")
            print(" ")

            '''
            print("Dataframe solo inner index")
            print(df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d])
            '''
            print("solo leyes")
            print(df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d].iloc[[4]])
            print("\n")

            sub_df=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d].iloc[[4]].T
            print("\n")        
            #print(sub_df)
            print("\n")

            sub_df["percent_ley"] = (sub_df.iloc[:,0] / sub_df.iloc[:,0].sum()) 
            print("\n")
            print(sub_df)
            print("\n")
            print("  Umbrales",df_parametros.Valor[5])
            print("\n")
            print("ley promedio")
            print(sub_df.iloc[:,0].mean())
            print("\n")
            print("relativo aportado por el composito ",sub_df.iloc[-1,1]*100,"%")  ## procentaje del composito
            print(" ")
            print(" ")      

            df_CLAG_MR_2022.iloc[obs,22]=len(index_inner_3d)
            df_CLAG_MR_2022.iloc[obs,20]=sub_df.iloc[:,0].mean()
            df_CLAG_MR_2022.iloc[obs,23]=i

            df_CLAG_MR_2022.iloc[obs,21]=sub_df.iloc[-1,1]

            df_CLAG_MR_2022.iloc[obs,24]=1                 #### DUMMY 



            

            print("pre_bucle")
            
            ley_sin=sub_df.iloc[:-1,1].mean()      ### promedio SIN composito central
            ley_con=sub_df.iloc[:,1].mean()        ### promedio CON composito central
            variax=abs((ley_con/ley_sin)-1)

            if variax > df_parametros.Valor[5][0]:
                print("T1")
                print(df_parametros.Valor[5][0])
                df_CLAG_MR_2022.iloc[obs,8]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][0])
                df_CLAG_MR_2022.iloc[obs,9]=1             
                if variax > df_parametros.Valor[5][1]:
                    print("T2")
                    print(df_parametros.Valor[5][1])
                    df_CLAG_MR_2022.iloc[obs,10]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][1])
                    df_CLAG_MR_2022.iloc[obs,11]=1         
                    if variax> df_parametros.Valor[5][2]:
                        print("T3")
                        print(df_parametros.Valor[5][2])
                        df_CLAG_MR_2022.iloc[obs,12]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][2])
                        df_CLAG_MR_2022.iloc[obs,13]=1
                        if variax > df_parametros.Valor[5][3]:
                            print("T4")
                            print(df_parametros.Valor[5][3])
                            df_CLAG_MR_2022.iloc[obs,14]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][3])
                            df_CLAG_MR_2022.iloc[obs,15]=1
                            if variax > df_parametros.Valor[5][4]:
                                print("T5")
                                print(df_parametros.Valor[5][4])
                                df_CLAG_MR_2022.iloc[obs,16]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][4])
                                df_CLAG_MR_2022.iloc[obs,17]=1
                                if variax > df_parametros.Valor[5][5]:
                                    print("T6")
                                    print(df_parametros.Valor[5][5])
                                    df_CLAG_MR_2022.iloc[obs,18]=ley-(sub_df.iloc[:,0].mean()*df_parametros.Valor[5][5])
                                    df_CLAG_MR_2022.iloc[obs,19]=1               
            else:
                print("NO EN UMBRAL MINIMO")


            

            freno_1_composito_x_roca=freno_1_composito_x_roca+1
            if freno_1_composito_x_roca>0:
                break

   
        freno_2_roca_x_zona=freno_2_roca_x_zona+1
        if freno_2_roca_x_zona>0:
            break
     
    freno_3_zona=freno_3_zona+1
    if freno_3_zona>0:
        break
    iterador=iterador+1


    print(" ")
    print(" ")



#df_CLAG_MR_2022.to_csv("Local_ag_martes_test.csv")






zone: 41
 
azimuth: 140
cartesiana: 310
radian: 5.410520681182422
ratio anisotropia: 2.5




    rock: 32
    zone: 41
 
    parametros de la Roca 
    ratio anisotropia: 2.5
    azimuth:  140
    cartesiana:  310
    radian:  5.410520681182422


        index composito : 197
 
        x:  3724.84
        y:  4797.7
        z:  4196.19
 
        holeid:  A080
        ley composito :  95.25
 
 
            expansion:  0
 
             []
            longitud index_inner:    0
 
 
            expansion:  1
 
             [3577]
            longitud index_inner:    1
 
 
            expansion:  2
 
             [3577, 3578, 3579]
            longitud index_inner:    3
 
 
            expansion:  3
 
             [3577, 3578, 3579]
            longitud index_inner:    3
 
 
            expansion:  4
 
             [3577, 3578, 3579, 1905, 1906, 1907, 1908]
            longitud index_inner:    7
 
 
            expansion:  5
 
             [3577, 3578, 3579, 1905, 1906, 1907, 1908, 1900,

In [21]:
composito_counts

501

In [28]:
composito_counts

1

In [22]:
df_CLAG_MR_2022.T[73]

COUNT                 6586
mid_x            3185.3567
mid_y            4329.8712
mid_z            4116.0064
Ag_ppm_BESTEL     0.828571
NBKRT                   31
AG_SUBZONAS             41
HOLEID                A055
CAPVAL1_PY        0.828571
CAPPED1_PY               0
CAPVAL2_PY        0.828571
CAPPED2_PY               0
CAPVAL3_PY        0.828571
CAPPED3_PY               0
CAPVAL4_PY        0.828571
CAPPED4_PY               0
CAPVAL5_PY        0.828571
CAPPED5_PY               0
CAPVAL6_PY        0.828571
CAPPED6_PY               0
AvgSin_PY        14.745175
PROP_12_PY        0.002676
Num_Data_PY             21
EXPAN_PY                 8
EVALUADO_PY              1
Name: 73, dtype: object

In [12]:
df_CLAG_MR_2022

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1_PY,CAPPED1_PY,CAPVAL2_PY,CAPPED2_PY,CAPVAL3_PY,CAPPED3_PY,CAPVAL4_PY,CAPPED4_PY,CAPVAL5_PY,CAPPED5_PY,CAPVAL6_PY,CAPPED6_PY,AvgSin_PY,PROP_12_PY,Num_Data_PY,EXPAN_PY,EVALUADO_PY
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0.0,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0.0,0,0,0,0
2,2338,4074.6121,4847.5252,4194.2584,35.799999,31,42,A019,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,0.0,0,0,0,0
3,2339,4075.6575,4846.2982,4191.7282,44.693333,31,42,A019,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,0.0,0,0,0,0
4,2340,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,4234.5329,5190.9378,4185.3258,7.000000,31,42,A913,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,0.0,0,0,0,0
4251,390970,4234.9222,5190.5647,4181.7917,7.457831,31,42,A913,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,0.0,0,0,0,0
4252,390971,4235.3126,5190.1954,4178.2573,8.333333,31,42,A913,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,0.0,0,0,0,0
4253,394849,3690.3063,4739.9698,4152.7452,122.400002,32,41,A951,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,0.0,0,0,0,0


In [16]:
sub_df

,Ag_ppm_BESTEL,percent_ley
377,9.0,0.025405
378,4.6,0.012985
3353,14.733334,0.041588
3354,13.596774,0.03838
369,18.333334,0.05175
370,11.266666,0.031803
371,7.0,0.019759
372,29.75,0.083976
1837,6.151515,0.017364
1838,16.200001,0.045728


In [24]:
ag_df=pd.read_csv("C:\FORTRAN\clsilver_mr20221.out",sep=",")
## obs
#ag_tt=ag_df[ag_df.ID==2336]
ag_df=ag_df.rename(columns={"ID": "COUNT"})
ag_df[ag_df.CAPPED1==1]
#df_1=ag_df[ag_df.CAPPED1==1]
df_1=ag_df

df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1_PY==1]#[:1]
df_2=df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1_PY==1]#[:1]

df_cap1 = pd.merge(df_1, df_2, on='COUNT')
df_cap1

,COUNT,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1_PY,CAPPED1_PY,CAPVAL2_PY,CAPPED2_PY,CAPVAL3_PY,CAPPED3_PY,CAPVAL4_PY,CAPPED4_PY,CAPVAL5_PY,CAPPED5_PY,CAPVAL6_PY,CAPPED6_PY,AvgSin_PY,PROP_12_PY,Num_Data_PY,Expan_PY
0,2340,A019,4076.70,4845.07,4189.20,31,42.0,45.19333,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,45.19333,0,25.5051,0.176,21,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019,43.622139,1,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,0,0,0,0
1,2845,A023,4008.81,4924.67,4193.37,31,42.0,33.59762,33.59762,0,33.59762,0,33.59762,0,33.59762,0,33.59762,0,33.59762,0,12.8891,1.000,21,4008.8100,4924.6700,4193.3700,33.597618,31,42,A023,32.606644,1,32.358900,1,33.597618,0,33.597618,0,33.597618,0,33.597618,0,0,0,0,0
2,2853,A023,4008.81,4924.67,4169.77,31,42.0,43.29000,40.27166,1,43.29000,0,43.29000,0,43.29000,0,43.29000,0,43.29000,0,12.5849,1.000,21,4008.8100,4924.6700,4169.7700,43.289997,31,42,A023,42.138143,1,41.850179,1,41.562215,1,43.289997,0,43.289997,0,43.289997,0,0,0,0,0
3,6606,A055,3216.48,4296.33,4077.20,32,41.0,88.40500,79.37804,1,88.40500,0,88.40500,0,88.40500,0,88.40500,0,88.40500,0,24.8056,0.476,21,3216.4829,4296.3285,4077.1986,88.404999,32,41,A055,85.563272,1,88.404999,0,88.404999,0,88.404999,0,88.404999,0,88.404999,0,0,0,0,0
4,6623,A055,3241.57,4269.29,4043.69,32,41.0,80.60667,79.37804,1,80.60667,0,80.60667,0,80.60667,0,80.60667,0,80.60667,0,24.8056,0.714,21,3241.5722,4269.2913,4043.6879,80.606667,32,41,A055,78.328493,1,77.758950,1,80.606667,0,80.606667,0,80.606667,0,80.606667,0,0,0,0,0
5,8060,A070,3580.88,4787.66,4130.86,31,41.0,89.20000,89.20000,0,89.20000,0,89.20000,0,89.20000,0,89.20000,0,89.20000,0,53.9795,0.182,21,3580.8800,4787.6600,4130.8600,89.199997,31,41,A070,86.151861,1,89.199997,0,89.199997,0,89.199997,0,89.199997,0,89.199997,0,0,0,0,0
6,8061,A070,3580.88,4787.66,4127.86,31,41.0,102.70000,97.08445,1,102.70000,0,102.70000,0,102.70000,0,102.70000,0,102.70000,0,30.3389,0.138,21,3580.8800,4787.6600,4127.8600,102.699997,31,41,A070,99.831268,1,99.114085,1,102.699997,0,102.699997,0,102.699997,0,102.699997,0,0,0,0,0
7,8077,A070,3580.88,4787.66,4079.86,31,41.0,99.15000,99.15000,0,99.15000,0,99.15000,0,99.15000,0,99.15000,0,99.15000,0,33.0471,0.875,21,3580.8800,4787.6600,4079.8600,99.150002,31,41,A070,96.390764,1,95.700954,1,99.150002,0,99.150002,0,99.150002,0,99.150002,0,0,0,0,0
8,8078,A070,3580.88,4787.66,4076.86,31,41.0,95.08333,95.08333,0,95.08333,0,95.08333,0,95.08333,0,95.08333,0,95.08333,0,33.0471,0.778,21,3580.8800,4787.6600,4076.8600,95.083328,31,41,A070,92.490868,1,91.842753,1,91.194638,1,95.083328,0,95.083328,0,95.083328,0,0,0,0,0
9,9335,A080,3724.84,4797.70,4199.19,32,41.0,59.15000,59.15000,0,59.15000,0,59.15000,0,59.15000,0,59.15000,0,59.15000,0,26.4095,0.099,21,3724.8400,4797.7000,4199.1900,59.150002,32,41,A080,57.677718,1,57.309647,1,56.941576,1,56.573505,1,59.150002,0,59.150002,0,0,0,0,0


In [25]:
df_cap1 = df_cap1[["COUNT","VALUE",
"CAPVAL1","CAPPED1","CAPVAL1_PY","CAPPED1_PY",
"CAPVAL2","CAPPED2","CAPVAL2_PY","CAPPED2_PY",
"CAPVAL3","CAPPED3","CAPVAL3_PY","CAPPED3_PY",
"CAPVAL4","CAPPED4","CAPVAL4_PY","CAPPED4_PY",
"CAPVAL5","CAPPED5","CAPVAL5_PY","CAPPED5_PY",
"CAPVAL6","CAPPED6","CAPVAL6_PY","CAPPED6_PY",
"AvgSin","AvgSin_PY",	
"PROP_12","PROP_12_PY",
"Num_Data","Num_Data_PY"]]
df_cap1

,COUNT,VALUE,CAPVAL1,CAPPED1,CAPVAL1_PY,CAPPED1_PY,CAPVAL2,CAPPED2,CAPVAL2_PY,CAPPED2_PY,CAPVAL3,CAPPED3,CAPVAL3_PY,CAPPED3_PY,CAPVAL4,CAPPED4,CAPVAL4_PY,CAPPED4_PY,CAPVAL5,CAPPED5,CAPVAL5_PY,CAPPED5_PY,CAPVAL6,CAPPED6,CAPVAL6_PY,CAPPED6_PY,AvgSin,AvgSin_PY,PROP_12,PROP_12_PY,Num_Data,Num_Data_PY
0,2340,45.19333,45.19333,0,43.622139,1,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,45.19333,0,45.193333,0,25.5051,0,0.176,0,21,0
1,2845,33.59762,33.59762,0,32.606644,1,33.59762,0,32.358900,1,33.59762,0,33.597618,0,33.59762,0,33.597618,0,33.59762,0,33.597618,0,33.59762,0,33.597618,0,12.8891,0,1.000,0,21,0
2,2853,43.29000,40.27166,1,42.138143,1,43.29000,0,41.850179,1,43.29000,0,41.562215,1,43.29000,0,43.289997,0,43.29000,0,43.289997,0,43.29000,0,43.289997,0,12.5849,0,1.000,0,21,0
3,6606,88.40500,79.37804,1,85.563272,1,88.40500,0,88.404999,0,88.40500,0,88.404999,0,88.40500,0,88.404999,0,88.40500,0,88.404999,0,88.40500,0,88.404999,0,24.8056,0,0.476,0,21,0
4,6623,80.60667,79.37804,1,78.328493,1,80.60667,0,77.758950,1,80.60667,0,80.606667,0,80.60667,0,80.606667,0,80.60667,0,80.606667,0,80.60667,0,80.606667,0,24.8056,0,0.714,0,21,0
5,8060,89.20000,89.20000,0,86.151861,1,89.20000,0,89.199997,0,89.20000,0,89.199997,0,89.20000,0,89.199997,0,89.20000,0,89.199997,0,89.20000,0,89.199997,0,53.9795,0,0.182,0,21,0
6,8061,102.70000,97.08445,1,99.831268,1,102.70000,0,99.114085,1,102.70000,0,102.699997,0,102.70000,0,102.699997,0,102.70000,0,102.699997,0,102.70000,0,102.699997,0,30.3389,0,0.138,0,21,0
7,8077,99.15000,99.15000,0,96.390764,1,99.15000,0,95.700954,1,99.15000,0,99.150002,0,99.15000,0,99.150002,0,99.15000,0,99.150002,0,99.15000,0,99.150002,0,33.0471,0,0.875,0,21,0
8,8078,95.08333,95.08333,0,92.490868,1,95.08333,0,91.842753,1,95.08333,0,91.194638,1,95.08333,0,95.083328,0,95.08333,0,95.083328,0,95.08333,0,95.083328,0,33.0471,0,0.778,0,21,0
9,9335,59.15000,59.15000,0,57.677718,1,59.15000,0,57.309647,1,59.15000,0,56.941576,1,59.15000,0,56.573505,1,59.15000,0,59.150002,0,59.15000,0,59.150002,0,26.4095,0,0.099,0,21,0


In [26]:
df_cap1.to_csv("EXPORT_TEST_FFDS.csv")
		

In [114]:
df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1==1].COUNT
for x in df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1==1].COUNT:
    print(x)

6606
8060
8061
8077
8078


In [121]:
ag_df=pd.read_csv("C:\FORTRAN\clsilver_mr20221.out",sep=",")
ag_df

,ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,A019,4072.29,4850.25,4199.87,32,42.0,25.14651,25.14651,0,...,0,25.14651,0,25.14651,0,25.14651,0,30.6503,0.231,21
1,2337,A019,4073.57,4848.75,4196.79,31,42.0,40.00000,40.00000,0,...,0,40.00000,0,40.00000,0,40.00000,0,25.7654,0.213,21
2,2338,A019,4074.61,4847.53,4194.26,31,42.0,35.80000,35.80000,0,...,0,35.80000,0,35.80000,0,35.80000,0,26.4273,0.196,21
3,2339,A019,4075.66,4846.30,4191.73,31,42.0,44.69333,44.69333,0,...,0,44.69333,0,44.69333,0,44.69333,0,25.8114,0.179,21
4,2340,A019,4076.70,4845.07,4189.20,31,42.0,45.19333,45.19333,0,...,0,45.19333,0,45.19333,0,45.19333,0,25.5051,0.176,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,A913,4234.53,5190.94,4185.33,31,42.0,7.00000,7.00000,0,...,0,7.00000,0,7.00000,0,7.00000,0,15.1952,0.125,21
4251,390970,A913,4234.92,5190.56,4181.79,31,42.0,7.45783,7.45783,0,...,0,7.45783,0,7.45783,0,7.45783,0,15.1952,0.119,21
4252,390971,A913,4235.31,5190.20,4178.26,31,42.0,8.33333,8.33333,0,...,0,8.33333,0,8.33333,0,8.33333,0,15.1952,0.163,21
4253,394849,A951,3690.31,4739.97,4152.75,32,41.0,122.40000,122.40000,0,...,0,122.40000,0,122.40000,0,122.40000,0,46.7659,0.000,21


In [119]:
df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1==1].COUNT

93     6606
151    8060
152    8061
168    8077
169    8078
Name: COUNT, dtype: int64

In [125]:
ag_df[ag_df.ID==6606]

,ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
93,6606,A055,3216.48,4296.33,4077.2,32,41.0,88.405,79.37804,1,88.405,0,88.405,0,88.405,0,88.405,0,88.405,0,24.8056,0.476,21


In [127]:
df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==6606]

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
93,6606,3216.4829,4296.3285,4077.1986,88.404999,32,41,A055,31.215749,1,88.404999,0,88.404999,0,88.404999,0,88.404999,0,88.404999,0,0,0,0


In [155]:
df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1==1].COUNT

93     6606
151    8060
152    8061
168    8077
169    8078
Name: COUNT, dtype: int64

In [165]:
n=8078

In [166]:
ag_df[ag_df.ID==n].iloc[:,8:]

,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
169,95.08333,0,95.08333,0,95.08333,0,95.08333,0,95.08333,0,95.08333,0,33.0471,0.778,21


In [167]:
df_CLAG_MR_2022[df_CLAG_MR_2022.COUNT==n].iloc[:,8:]

,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
169,30.54873,1,95.083328,0,95.083328,0,95.083328,0,95.083328,0,95.083328,0,0,0,0


In [152]:
ag_df[ag_df.ID==n].iloc[0,8]-88.405

-9.026960000000003

In [153]:
df_CLAG_MR_2022.T[obs][22]

0

In [154]:
df_CLAG_MR_2022.T[obs]

COUNT                 6618
mid_x            3234.6872
mid_y            4276.7109
mid_z            4053.5034
Ag_ppm_BESTEL        3.052
NBKRT                   32
AG_SUBZONAS             41
HOLEID                A055
CAPVAL1              3.052
CAPPED1                  0
CAPVAL2              3.052
CAPPED2                  0
CAPVAL3              3.052
CAPPED3                  0
CAPVAL4              3.052
CAPPED4                  0
CAPVAL5              3.052
CAPPED5                  0
CAPVAL6              3.052
CAPPED6                  0
AvgSin                   0
PROP_12                  0
Num_Data                 0
Name: 105, dtype: object

In [ ]:
ag_df=pd.read_csv("C:\FORTRAN\clsilver_mr20221.out",sep=",")
## obs
#ag_tt=ag_df[ag_df.ID==2336]
ag_df=ag_df.rename(columns={"ID": "COUNT"})
ag_df[ag_df.CAPPED1==1]
#df_1=ag_df[ag_df.CAPPED1==1]
df_1=ag_df

df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1_PY==1]#[:1]
df_2=df_CLAG_MR_2022[df_CLAG_MR_2022.CAPPED1_PY==1]#[:1]

df_cap1 = pd.merge(df_1, df_2, on='COUNT')
df_cap1

In [27]:
df_CLAG_MR_2022

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1_PY,CAPPED1_PY,CAPVAL2_PY,CAPPED2_PY,CAPVAL3_PY,CAPPED3_PY,CAPVAL4_PY,CAPPED4_PY,CAPVAL5_PY,CAPPED5_PY,CAPVAL6_PY,CAPPED6_PY,AvgSin_PY,PROP_12_PY,Num_Data_PY,Expan_PY
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,25.146513,0,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,40.000000,0,0,0,0,0
2,2338,4074.6121,4847.5252,4194.2584,35.799999,31,42,A019,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,35.799999,0,0,0,0,0
3,2339,4075.6575,4846.2982,4191.7282,44.693333,31,42,A019,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,44.693333,0,0,0,0,0
4,2340,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019,43.622139,1,45.193333,0,45.193333,0,45.193333,0,45.193333,0,45.193333,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,4234.5329,5190.9378,4185.3258,7.000000,31,42,A913,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,7.000000,0,0,0,0,0
4251,390970,4234.9222,5190.5647,4181.7917,7.457831,31,42,A913,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,7.457831,0,0,0,0,0
4252,390971,4235.3126,5190.1954,4178.2573,8.333333,31,42,A913,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,8.333333,0,0,0,0,0
4253,394849,3690.3063,4739.9698,4152.7452,122.400002,32,41,A951,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,122.400002,0,0,0,0,0


In [31]:
df_CLAG_MR_2022.T[obs][22]=len(index_inner_3d)



In [32]:
df_CLAG_MR_2022.T[obs][22]


0

In [33]:
df_CLAG_MR_2022.iloc[obs,22]

0

In [ ]:
from pyspark import SparkContext

# Crear un contexto de Spark
sc = SparkContext("local", "Paralelizar bucle for con PySpark")

# Crear una lista de datos
data = range(100)

# Convertir la lista en un RDD de Spark
rdd_data = sc.parallelize(data)

# Definir la función que se ejecutará en cada iteración del bucle
def mi_funcion(x):
    return x * 2

# Aplicar la función a cada elemento del RDD usando la función "map"
resultado = rdd_data.map(mi_funcion).collect()

# Imprimir los resultados
print(resultado)

# Detener el contexto de Spark
sc.stop()

In [25]:



iterador=0      ##  PARA SIGNAR RATIO x ZONA

composito_counts=0
sub_composito_counts=[]

flag_holeid=df_parametros.Valor[15][0]

#for zone in df_parametros.Valor[12][:1]:     ## borrar   [:2]
zonas = df_parametros.Valor[12]
for zone in zonas:     #################################  FIJO PARA PRUEBA  BORRAR
    
    print("\n\n")
    print("zone:",zone)
    print(" ") 
    #  df_parametros.Valor[13]         ## LISTA DE AZIMUTH
    print("azimuth:",df_parametros.Valor[13][iterador])
    azimuth=df_parametros.Valor[13][iterador]
    cartesiana=0          ## <------ 

    if azimuth < 90:
        cartesiana=90-azimuth 
    elif azimuth < 180:
        cartesiana=360-(azimuth-90) 
    elif azimuth < 270:
        cartesiana=270-(azimuth-180) 
    elif azimuth < 360:
        cartesiana=180-(azimuth-270) 
        
    print("cartesiana:",cartesiana)
    rad=math.radians(cartesiana)    ## <------  APLICAR ACA SIN COS etc etc    
    print("radian:",rad)
    #df_parametros.Valor[14]                            ## LISTA DE ANISOTROPIA 
    print("ratio anisotropia:",df_parametros.Valor[14][iterador])


    rocks=df_parametros.Valor[8]
    for rock in rocks:

        freno_1_composito_x_roca=0

        print("\n\n\n")
        print("    rock:",rock)
        print("    zone:",zone)
        print(" ")
        print("    parametros de la Roca ")
        print("    ratio anisotropia:",ratio)
        print("    azimuth: ",azimuth)
        print("    cartesiana: ",cartesiana)
        print("    radian: ",rad)

        for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:  ## ITERAR INDICES

            composito_counts=composito_counts+1

            print("\n")
            print("        index composito :",obs)

        sub_composito_counts.append(composito_counts)











zone: 41
 
azimuth: 140
cartesiana: 310
radian: 5.410520681182422
ratio anisotropia: 2.5




    rock: 31
    zone: 41
 
    parametros de la Roca 
    ratio anisotropia: 2.5
    azimuth:  140
    cartesiana:  310
    radian:  5.410520681182422


        index composito : 73


        index composito : 74


        index composito : 75


        index composito : 76


        index composito : 77


        index composito : 116


        index composito : 117


        index composito : 118


        index composito : 119


        index composito : 120


        index composito : 121


        index composito : 151


        index composito : 152


        index composito : 153


        index composito : 154


        index composito : 155


        index composito : 156


        index composito : 157


        index composito : 158


        index composito : 159


        index composito : 160


        index composito : 161


        index composito : 162


        index compo

In [26]:
sub_composito_counts

[1089, 1701, 3214, 4255]

In [30]:
sub_composito_counts[1]-sub_composito_counts[0]

612

In [25]:
sub_df.iloc[:,0]

3142         18.0
2539    13.382353
3636         12.0
3637         10.0
3638         26.0
1288         16.0
1289         25.0
1290    34.333332
1330          5.0
1331         42.0
1332         57.0
1333         14.5
1334         12.0
1335         13.0
1336    27.566668
1337         29.0
1338    18.799999
1339          5.0
1585     7.983333
1586     2.383333
1            40.0
Name: Ag_ppm_BESTEL, dtype: object

In [26]:
sub_df

,Ag_ppm_BESTEL,percent_ley
3142,18.0,0.041963
2539,13.382353,0.031198
3636,12.0,0.027975
3637,10.0,0.023313
3638,26.0,0.060613
1288,16.0,0.0373
1289,25.0,0.058282
1290,34.333332,0.080041
1330,5.0,0.011656
1331,42.0,0.097914
